In [1]:
# [Cell 1: .env 로드 및 Agent 임포트]
# .env 파일 로드 (core.py가 로드하지만, 노트북 환경을 위해 한 번 더)
import sys
import os
from dotenv import load_dotenv

# 1. 현재 프로젝트 루트 디렉토리(jesafe/)의 절대 경로를 찾습니다.
# os.getcwd() (현재 폴더)의 부모 폴더('..')가 프로젝트 루트라고 지정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. 이 경로를 파이썬 모듈 검색 경로(sys.path)에 추가합니다.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"프로젝트 루트 경로 추가: {project_root}")

load_dotenv()
print(".env 로드 완료:", os.getenv("DATABASE_URL") is not None)

프로젝트 루트 경로 추가: /Users/lwj0831/jesafe
.env 로드 완료: True


In [ ]:
# [Cell 2: 메인 Agent 동기 테스트 (RunnableWithMessageHistory 사용)]

from src.services import main_agent_executor
from src.data_loader import _load_jeju_coords

# 테스트 전에 CSV 데이터를 메모리에 로드합니다.
_load_jeju_coords()

# 테스트에 사용할 고유 세션 ID
session_id = "test_session_notebook_123"

# --- 1. 날씨 질문 테스트 ---
# (프롬프트 규칙에 따라 '제주도'가 '제주시'로 변환되어야 함)
query1 = "오늘 제주도 날씨 어때?"
print(f"User: {query1}")

response1 = main_agent_executor.invoke(
    {"input": query1},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response1['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 2. 사고 통계 질문 테스트 ---
query2 = "제주시에서 낙상 사고가 몇 건이야?"
print(f"User: {query2}")

response2 = main_agent_executor.invoke(
    {"input": query2},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response2['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 3. (중요) 대화 맥락(Context) 질문 테스트 ---
# (이전 질문의 '제주시'를 기억하고 '제주시' 날씨를 답해야 함)
query3 = "그럼 그 지역 날씨는 어때?"
print(f"User: {query3}")

response3 = main_agent_executor.invoke(
    {"input": query3},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response3['output']}\n")

In [ ]:
# [Cell 3: RAG 임베딩 및 리트리버 전체 테스트]
import sys
import os
# (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.core import get_vector_store, get_bm25_retriever
from src.core import _compression_retriever_instance

print("Connecting to Vector Store & Initializing Retrievers...")
try:
    # 앱 시작 시점과 동일하게 리소스 초기화
    vector_store = get_vector_store()
    bm25_retriever = get_bm25_retriever()
    
    # 테스트 쿼리
    query = "제주도 오름 추천해줘"
    print(f"\nTest Query: '{query}'")
    
    # -----------------------------------------------
    # 테스트 1: PGVector (의미기반 검색)
    # -----------------------------------------------
    print("\n--- 1. PGVector (Similarity Search) Test ---")
    vector_results = vector_store.similarity_search(query, k=3)
    print(f"Found {len(vector_results)} Chunks:")
    for i, doc in enumerate(vector_results):
        print(f"  [Vec {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 2: BM25 (키워드 기반 검색)
    # -----------------------------------------------
    print("\n--- 2. BM25 (Keyword Search) Test ---")
    bm25_results = bm25_retriever.invoke(query) 
    print(f"Found {len(bm25_results)} Chunks:")
    for i, doc in enumerate(bm25_results):
        print(f"  [BM25 {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 3: Ensemble (최종 앙상블)
    # -----------------------------------------------
    print("\n--- 3. Ensemble Retriever Test ---")
    ensemble_results = _compression_retriever_instance.base_retriever.invoke(query)
    print(f"Found {len(ensemble_results)} Chunks (after re-ranking):")
    for i, doc in enumerate(ensemble_results):
        print(f"  [Ensemble {i+1}] (Source: {doc.metadata.get('source', 'N/A')})")
        print(f"    {doc.page_content[:200]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

In [11]:
# [Cell 4: EmbeddingsFilter 상세 디버깅]
import sys
import os
import numpy as np
from langchain_community.utils.math import cosine_similarity

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. src/core.py에서 핵심 컴포넌트 임포트
from src.core import get_compression_retriever, get_cached_embedder

# 3. 테스트 쿼리 정의
query = "고사리 육개장"
similarity_threshold = 0.80 # ⬅️ core.py에 설정된 임계값

print(f"Query: {query}")
print(f"Similarity Threshold: {similarity_threshold}")
print("-" * 30)

try:
    # 4. 리트리버 및 임베더 로드
    compression_retriever = get_compression_retriever()
    # 4-1. 압축 전 '기본' 리트리버 (EnsembleRetriever)
    base_retriever = compression_retriever.base_retriever
    # 4-2. 점수 계산에 사용할 임베더
    embedder = get_cached_embedder()

    # 5. (압축 전) 앙상블 리트리버가 찾은 문서 원본 가져오기
    print("Fetching docs from EnsembleRetriever (before filtering)...")
    retrieved_docs = base_retriever.invoke(query)
    
    if not retrieved_docs:
        print("EnsembleRetriever가 문서를 찾지 못했습니다.")
    else:
        print(f"Found {len(retrieved_docs)} docs. Calculating similarity scores...")
        
        # 6. (핵심) 쿼리와 각 문서의 임베딩 점수 수동 계산
        query_embedding = embedder.embed_query(query)
        doc_embeddings = embedder.embed_documents([doc.page_content for doc in retrieved_docs])
        
        # 7. 코사인 유사도 계산
        scores = cosine_similarity([query_embedding], doc_embeddings)[0]
        
        # 8. 결과 출력
        for i, (doc, score) in enumerate(zip(retrieved_docs, scores)):
            is_passed = score >= similarity_threshold
            
            print(f"\n[Doc {i+1}] Score: {score:.4f} (Threshold: {similarity_threshold})")
            print(f"  -> PASSED: {is_passed} ⬅️")
            print(f"  -> Content: {doc.page_content[:150]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

Query: 고사리 육개장
Similarity Threshold: 0.8
------------------------------
Fetching docs from EnsembleRetriever (before filtering)...
Found 8 docs. Calculating similarity scores...

[Doc 1] Score: 0.8522 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: # 오설록티뮤지엄
- 서귀포시 안덕면 신화역사로 15  
국내외 차 관련 물품과 박물관이며, 푸른 녹차밭이 펼쳐져 있는 풍경으로 유명. 티 하우스에서는 티 마스터가 직접 볶은 따뜻한 차와 오설록 녹차로 만든 음료 아이스크림, 롤케이크 등을 즐길 수 있다.





Tr...

[Doc 2] Score: 0.8462 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: # 섭지코지
- 📍 서귀포시 성산읍 고성리 62-3  
드라마 촬영지로 유명한 곳. 봄에는 노란 유채꽃과 성산일출봉을 배경으로 한 해안절경이 장관이고, 가을에는 잔디에 돋아난 들꽃들과 은빛 물결의 억새가 로맨틱한 분위기를 자아낸다.

# 성산일출봉
- 📍 서귀포시 성산...

[Doc 3] Score: 0.8561 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: 깅이죽 이미지





# 저지방 고단백, 불포화지방산  
## 말고기

저지방 고단백 저칼로리 웰빙 식품으로 각광받고 있으며 미오글로빈 함량이 높아  
육색부터 소고기와 다르다. 지방이 적어 육회로 먹기에 적당하며 철분 함량이 다른  
육류에 비해 높아 빈혈 환자에게...

[Doc 4] Score: 0.8472 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: # 갈치구이
싱싱한 은갈치를 내장만을 제거하고 등에 고르게 칼집을 넣어 굵은소금을 뿌린 후 굽는다. 싱싱한 

In [ ]:
# [Cell 5: DDGS 웹 검색 결과 수동 테스트]
import sys
import os

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. ddgs 라이브러리 직접 임포트
try:
    from ddgs import DDGS
except ImportError:
    print("❌ 'ddgs' 패키지가 설치되지 않았습니다. 'poetry add ddgs'를 실행하세요.")

# 3. 테스트 쿼리
query = "제주도 7월 축제"

print(f"Testing DDGS() with query: '{query}'")
print(f"Requesting max_results=3...")
print("-" * 30)

try:
    # 4. src/tools.py의 핵심 로직 직접 실행
    # (DDGS()는 컨텍스트 매니저(with) 사용을 권장)
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
    
    # 5. 결과 확인
    if not results:
        print("❌ 검색 결과가 없습니다.")
    else:
        print(f"✅ {len(results)}개의 결과를 성공적으로 가져왔습니다:")
        
        for i, res in enumerate(results):
            print(f"\n[Result {i+1}]")
            print(f"  Title: {res.get('title')}")
            print(f"  Snippet: {res.get('body', '')[:150]}...")
            print(f"  URL: {res.get('href')}")

except Exception as e:
    print(f"❌ 테스트 실패: {e}")

Testing DDGS() with query: '제주도 7월 축제'
Requesting max_results=3...
------------------------------
✅ 3개의 결과를 성공적으로 가져왔습니다:

[Result 1]
  Title: 제주시, 제주도 , 대한민국 월별 날씨 | AccuWeather
  Snippet: Get the monthly weather forecast for 제주시, 제주도 , 대한민국, including daily high/low, historical averages, to help you plan ahead....
  URL: https://www.accuweather.com/ko/kr/jeju/224209/july-weather/224209

[Result 2]
  Title: 제주도 11 월 축제 한눈에 보기! | Brunch Story
  Snippet: 태진아와 워너원, AOA, 레드벨벳, EXID, 오마이걸, 모모랜드, 나인뮤지스, 여자친구 등 총 28팀이 참가하는 제주도 최대 K팝 행사로 기대를 모으고 있답니다. 제주종합경기장 주경기장. 제주특별자치도 제주시 서광로2길 24....
  URL: https://brunch.co.kr/@gorrajeju/298

[Result 3]
  Title: 중3 수학여행 장기자랑 제주도 서연 | TikTok
  Snippet: #수학여행 #08 #07 # 제주도 #ootd #fyp 제주도 수학여행 코디 아이디어와 팁. 여름 제주도 수학여행을 위한 스타일과 코디를 소개합니다....
  URL: https://www.tiktok.com/discover/중3-수학여행-장기자랑-제주도-서연


In [10]:
#!/usr/bin/env python3
"""
리트리버 성능 디버깅 스크립트
각 단계별 결과를 확인하여 관련성 높은 청크를 놓치는 문제를 파악합니다.
"""

import sys
from src.core import create_debug_query_chain

# 테스트 쿼리들
test_queries = [
    #"저는 제주 오름 생태 트레킹 코스를 짜는 사람인데요, 보목 자리돔축제 안내문을 보다가 섭섬이랑 지귀도, 문섬, 범섬이 한꺼번에 보인다고 적혀 있던데 혹시 '지귀도'가 어디 근처에 떠 있는 섬인지, 그리고 이 축제 코스 짤 때 지귀도 풍경을 어떻게 엮어서 설명하면 좋을지 헷갈리네요 지귀도 위치나 풍경에 대해 다시 한 번 정확히 알려주실 수 있을까요?",
    #"As a Jeju Eco-Trek Planner who wants to recommend local seasonal cuisine that complements eco-healing treks on the oreum, how is 갈치 characterized in Jeju during autumn, and what are the distinctive features and ingredients of the main 갈치 dishes such as 갈치국, 갈치조림, and 갈치구이 that I could highlight to trekkers seeking an authentic, seasonal food experience?"
    #"제주 한림 지역에서 옥돔을 뭐라고 부르나요?",
    "저 제주 자연 여행 준비하는데요, 현지 향토음식 중에 고사리육개장 어떤 음식인지, 어떻게 끓이고 맛이랑 특징 뭐 있는지 좀 길게 설명해줄 수 있어요?"
]

if __name__ == "__main__":
    print("\n🚀 리트리버 디버깅 테스트 시작\n")
    
    for query in test_queries:
        docs = create_debug_query_chain().invoke(query)
        
        # 원하는 답변이 포함되어 있는지 확인
        print("\n✅ 검증:")
        expected_keywords = ["지귀도","자리돔"]
        found = False
        
        for doc in docs:
            print("\n" + "-"*60)
            print(doc.page_content)
            print("\n" + "-"*60)
            for keyword in expected_keywords:
                if keyword in doc.page_content:
                    print(f"   ✓ 예상 키워드 '{keyword}' 발견!")
                    found = True
                    break
        
        if not found:
            print(f"   ✗ 예상된 청크를 찾지 못함")
        
        print("\n" + "-"*60 + "\n")


🚀 리트리버 디버깅 테스트 시작

🔹 한글 자연어 쿼리: 저 제주 자연 여행 준비하는데요, 현지 향토음식 중에 고사리육개장 어떤 음식인지, 어떻게 끓이고 맛이랑 특징 뭐 있는지 좀 길게 설명해줄 수 있어요?
   (BM25 내부 변환 미리보기: 제주 자연 여행 준비 는 데 요 현지 향토 음식 중 고사리 육개장 음식 ᆫ지 어떻 게 끓이 고 맛 이랑 특징 뭐 있 는지 길 게 설명 주 ᆯ 수 있 어요)
🔹 검색된 문서 수: 5
   1. ----  ### 제주7대 향토음식   제주도의 자연과 사회 문화적 환경을 배경으로 형성된 제주의 음식 문화...
   2. # 제주 7대 명품  ## 보물섬 제주도에 가면 꼭 봐야 할 곳, 꼭 맛봐야 할 것들은?   제주 7대 명품...
   3. Travel Jeju  # 제주여행 쇼핑리스트  ----  ## 제주감귤 제주의 대표 특산물 감귤. 달콤하고...

✅ 검증:

------------------------------------------------------------
----

### 제주7대 향토음식  
제주도의 자연과 사회 문화적 환경을 배경으로 형성된 제주의 음식 문화는 독특하다. 지리적으로 사면이 바다여서 사시사철 다양한 해산물과 어류를 이용한 음식이 많다.  
━  
* 자리돔물회  
* 갈치국  
* 성게국  
* 한치(오징어)물회  
* 옥돔구이  
* 빙떡  
* 고기국수

----

### 제주7대 특산물  
제주도의 특산물은 청정 푸른 바다와 비옥한 대지에서 생산 되기에 제주에 유일하거나 타 지역 특산물보다 우위의 품질을 자랑한다.  
━  
* 감귤  
* 고사리  
* 자리돔  
* 옥돔  
* 갈치  
* 말고기  
* 돼지고기

----

### 제주7대 건축  
세상에서 가장 아름다운 것은 자연이다. 그속에 자연을 거스르지 않고 조화를 이루면서 있는 제주건축은 또하나의 새로운 볼거리를 만들어 낸다.  
━  
* 제주현대미술관  
* 제주월드컵경기장  
* 다음커뮤니케이션 본사  
* 포

In [1]:
# [Cell 6: Cohere Reranker 통합 및 성능 검증 테스트] (수정됨)
import sys
import os

# 1. 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import create_debug_query_chain, debug_retriever_pipeline

print("🚀 Cohere Reranker 통합 테스트 시작...")

try:

    # 3. 테스트 쿼리
    query = "붉은 오름"
    
    # 4. 검색 실행
    final_docs = debug_retriever_pipeline(query)
    #final_docs = create_debug_query_chain().invoke({"question": query})
    
    if not final_docs:
        print("\n❌ 검색 결과가 없습니다. (DB 연결이나 임계값을 확인하세요)")
    else:
        print(f"\n✅ 최종 반환된 문서: {len(final_docs)}개")
        print("=" * 60)
        
        has_score = False
        
        for i, doc in enumerate(final_docs):
            score = doc.metadata.get('relevance_score', None)
            source = doc.metadata.get('source', 'Unknown')
            
            score_str = f"{score:.4f}" if score is not None else "N/A"
            if score is not None: has_score = True
            
            #content_preview = doc.page_content[:120].replace('\n', ' ')
            content_preview = doc.page_content
            
            print(f"[Rank {i+1}] Relevance Score: {score_str}")
            print(f"  Source: {source}")
            print(f"  Content Preview: {content_preview}...")
            print("-" * 60)
            
        # 5. 결과 검증 메시지
        print("\n[검증 결과]")
        if has_score:
            print("🎉 성공: 문서 메타데이터에서 'relevance_score'가 확인되었습니다.")
            print("   Cohere Reranker가 정상적으로 작동 중입니다.")
        else:
            print("⚠️ 주의: 'relevance_score'가 발견되지 않았습니다.")
            print("   CohereRerank가 파이프라인에 제대로 추가되지 않았거나, API 호출이 누락되었을 수 있습니다.")

except Exception as e:
    print(f"\n❌ 에러 발생: {e}")
    if "API" in str(e) or "key" in str(e).lower():
        print("💡 힌트: .env 파일에 COHERE_API_KEY가 올바르게 설정되었는지 확인하세요.")

🚀 Cohere Reranker 통합 테스트 시작...

🔍 디버깅: 원본 쿼리 = '붉은 오름'
📝 최적화된 쿼리 (자연어): 붉은 오름

[Stage 1] BM25 검색 (내부적으로 키워드 변환 수행)
Initializing BM25Retriever (from PGVector's stored documents)...
벡터스토어에서 문서 로드 중...
✅ 벡터스토어에서 148개 문서 로드 완료


Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


✅ BM25 Tokenizer 초기화 완료. 불용어 675개 적용됨.
BM25Retriever initialization complete.
  1. ### 드라마 ‘우리들의 블루스’ 촬영지   태웃개 TAEUSGAE   **태웃개 TAEUSGAE**   - **주소** 제주 서귀포시 남원읍 ...
  2. # 걷기 좋은 오름 * 원당봉 ............................................ 19   * 붉은오름 .........
  3. - 위치: 서귀포시 대정읍 상모리 산2   - 형태: 복합형   - 탐방시간: 2시간   - 주차장   - 화장실   - 주변여행: 산방산, 용...

[Stage 2] Vector 검색 (자연어 쿼리 사용)
Initializing PGVector connection to collection: tourism_docs...
Initializing CacheBackedEmbeddings...


/Users/lwj0831/jesafe/.venv/lib/python3.11/site-packages/langchain_classic/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


  1. # 걷기 좋은 오름 * 원당봉 ............................................ 19   * 붉은오름 .........
  2. 백약이오름은 백가지 약초가 자라다고 하여 붙여진 이름으로 봄, 여름,   가을 200여종의 제주들꽃들이 향연을 펼치는 곳이다. 성산일출봉의   ...
  3. # 오름이란?  ----  ## 오름이란?  한라산 정상의 백록담을 제외한 제주특별자치도 일원에 분포하며, 화구를 갖고 있으면서 화산분출물에 의...

[Stage 3] Ensemble 통합 결과
  1. # 걷기 좋은 오름 * 원당봉 ............................................ 19   * 붉은오름 .........
  2. 백약이오름은 백가지 약초가 자라다고 하여 붙여진 이름으로 봄, 여름,   가을 200여종의 제주들꽃들이 향연을 펼치는 곳이다. 성산일출봉의   ...
  3. - 위치: 서귀포시 대정읍 상모리 산2   - 형태: 복합형   - 탐방시간: 2시간   - 주차장   - 화장실   - 주변여행: 산방산, 용...

[Stage 4] 필터링 최종 결과
Initializing Compression Retriever (Ensemble + Filters)...
Compression Retriever initialization complete.
  1. - 위치: 서귀포시 대정읍 상모리 산2   - 형태: 복합형   - 탐방시간: 2시간   - 주차장   - 화장실   - 주변여행: 산방산, 용머리해안, 알뜨르비행장    ----...
  2. # 걷기 좋은 오름 * 원당봉 ............................................ 19   * 붉은오름 .............................
  3. 백약이오름은 백가지 약초가 자라다고 하여 붙여진 이름으로 봄, 여름,   가을 200여종의 제주들꽃들이 향연을 펼치는 곳이다. 성산일출봉의   일출과 한라산의 노

In [ ]:
from src.core import create_query_processing_chain


query = "돌하르방식당 뭐 파는 식당이고 어디 있는지 알려줄 수 있어요?"
chain = create_query_processing_chain()
result = chain.invoke({"question": query})
print(result)

In [5]:
# [Cell 7: BM25 토큰화 이슈 수정 검증 및 통합 테스트]
import sys
import os

# 1. 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import get_bm25_retriever, get_korean_bm25_tokenizer, debug_retriever_pipeline

print("🚀 BM25 수정 사항(래퍼 제거 및 직접 토큰화) 검증 테스트 시작...\n")

target_keyword = "고사리육개장"
query = "고사리 육개장"

# -----------------------------------------------------------
# 1. Tokenizer 직접 확인
# (BM25 내부에서 이 토크나이저가 돌 때 어떻게 쪼개는지 확인)
# -----------------------------------------------------------
print(f"1. Tokenizer Check for keyword: '{target_keyword}'")
tokenizer = get_korean_bm25_tokenizer()
tokens = tokenizer(target_keyword)
print(f"   👉 토큰화 결과: {tokens}")
print("   (참고: 이 결과가 ['고사리육개장'] 한 덩어리라면, 검색 시에도 한 덩어리로 매칭되어야 함)")


# -----------------------------------------------------------
# 2. Raw BM25 Retriever 단독 테스트
# (래퍼 없이 직접 invoke 했을 때 찾아오는지 확인)
# -----------------------------------------------------------
print(f"\n2. Raw BM25 Retriever Direct Test for '{query}'")
try:
    bm25 = get_bm25_retriever()
    bm25_docs = bm25.invoke(query)
    
    found_in_bm25 = False
    print(f"   🔍 BM25 반환 문서 수: {len(bm25_docs)}")
    
    for i, doc in enumerate(bm25_docs):
        # 줄바꿈 제거 후 미리보기
        content_preview = doc.page_content.replace('\n', ' ')[:80]
        print(f"   [{i+1}] {content_preview}...")
        
        # 핵심 키워드 포함 여부 체크
        if target_keyword in doc.page_content:
            print("      ✅ 타겟 문서 발견! (BM25가 정상 작동 중)")
            found_in_bm25 = True
            
    if not found_in_bm25:
        print(f"      ❌ 경고: BM25 상위 결과({len(bm25_docs)}개)에 '{target_keyword}' 문서가 없습니다.")

except Exception as e:
    print(f"   ❌ BM25 테스트 중 에러: {e}")


# -----------------------------------------------------------
# 3. Full Pipeline (Debug) 재확인
# (Ensemble -> Reranker 과정을 거쳐 최종 결과에 나오는지 확인)
# -----------------------------------------------------------
print(f"\n3. Full Pipeline Debug for '{query}'")
print("-" * 60)

# debug_retriever_pipeline 함수는 core.py에 정의된 것을 사용
final_docs = debug_retriever_pipeline(query)

print("\n[최종 검증]")
final_found = False
for doc in final_docs:
    if target_keyword in doc.page_content:
        final_found = True
        break

if final_found:
    print(f"🎉 성공: '{target_keyword}' 관련 문서가 최종 결과에 포함되었습니다!")
else:
    print(f"⚠️ 실패: 최종 결과에서 '{target_keyword}' 문서를 찾지 못했습니다.")

🚀 BM25 수정 사항(래퍼 제거 및 직접 토큰화) 검증 테스트 시작...

1. Tokenizer Check for keyword: '고사리육개장'
   👉 토큰화 결과: ['고사리', '육개장']
   (참고: 이 결과가 ['고사리육개장'] 한 덩어리라면, 검색 시에도 한 덩어리로 매칭되어야 함)

2. Raw BM25 Retriever Direct Test for '고사리 육개장'
   🔍 BM25 반환 문서 수: 5
   [1] * 몸국 ............................................ 18   * 고사리육개장 ...................
      ✅ 타겟 문서 발견! (BM25가 정상 작동 중)
   [2] 꼭 맛봐야 할 제주흑돼지 요리  # 고기국수 돼지잡뼈 우린 국물에 수육 삶은 국물을 섞어 도톰한 중면에 수육 듬뿍 얹어 먹는 국수다. 누린내가 ...
      ✅ 타겟 문서 발견! (BM25가 정상 작동 중)
   [3] * 자리물회 ........................................ 7   * 자리강회 ........................
      ✅ 타겟 문서 발견! (BM25가 정상 작동 중)
   [4] | 날래모슬포자리돔  | 자리물회, 몸국, 갈치조림              | 제주시 다랑곶 7길 21(노형동)    | 711-7465    ...
   [5] ----  ### 제주7대 향토음식   제주도의 자연과 사회 문화적 환경을 배경으로 형성된 제주의 음식 문화는 독특하다. 지리적으로 사면이 바다...

3. Full Pipeline Debug for '고사리 육개장'
------------------------------------------------------------

🔍 디버깅: 원본 쿼리 = '고사리 육개장'
📝 최적화된 쿼리 (자연어): 고사리 육개장

[Stage 1] BM25 검색 (내부적으로 키워드 변환 수행)
  1. * 몸국 .......

In [4]:
from src.core import get_korean_bm25_tokenizer

tokenizer = get_korean_bm25_tokenizer()

print(f"1. 붙여썼을 때: {tokenizer('고사리육개장')}")
# 예상 출력: ['고사리육개장'] (문서에 저장된 형태와 같음)

print(f"2. 띄어썼을 때: {tokenizer('고사리 육개장')}")
# 예상 출력: ['고사리', '육개장'] (문서에 저장된 형태와 다름 -> 검색 실패)

1. 붙여썼을 때: ['고사리', '육개장']
2. 띄어썼을 때: ['고사리', '육개장']


In [3]:
from src.core import get_bm25_retriever, load_documents_from_vectorstore

# 1. 저장된 문서 내용 확인
docs = load_documents_from_vectorstore()
print("\n=== 실제 저장된 문서 샘플 ===")
for i, doc in enumerate(docs[:5]):
    if '고사리' in doc.page_content or '육개장' in doc.page_content:
        print(f"{i}. {doc.page_content[:100]}")

# 2. BM25 retriever의 실제 검색 테스트
retriever = get_bm25_retriever()

print("\n=== '고사리육개장' 검색 ===")
results1 = retriever.invoke("고사리육개장")
for i, doc in enumerate(results1[:3]):
    print(f"{i+1}. Score: {doc.metadata.get('score', 'N/A')}")
    print(f"   Content: {doc.page_content[:80]}\n")

print("\n=== '고사리 육개장' 검색 ===")
results2 = retriever.invoke("고사리 육개장")
for i, doc in enumerate(results2[:3]):
    print(f"{i+1}. Score: {doc.metadata.get('score', 'N/A')}")
    print(f"   Content: {doc.page_content[:80]}\n")

벡터스토어에서 문서 로드 중...
✅ 벡터스토어에서 148개 문서 로드 완료

=== 실제 저장된 문서 샘플 ===

=== '고사리육개장' 검색 ===
1. Score: N/A
   Content: * 몸국 ............................................ 18  
* 고사리육개장 ................

2. Score: N/A
   Content: 꼭 맛봐야 할 제주흑돼지 요리

# 고기국수
돼지잡뼈 우린 국물에 수육 삶은 국물을 섞어 도톰한 중면에 수육 듬뿍 얹어 먹는 국수다. 누린내가 

3. Score: N/A
   Content: * 자리물회 ........................................ 7  
* 자리강회 .....................


=== '고사리 육개장' 검색 ===
1. Score: N/A
   Content: * 몸국 ............................................ 18  
* 고사리육개장 ................

2. Score: N/A
   Content: 꼭 맛봐야 할 제주흑돼지 요리

# 고기국수
돼지잡뼈 우린 국물에 수육 삶은 국물을 섞어 도톰한 중면에 수육 듬뿍 얹어 먹는 국수다. 누린내가 

3. Score: N/A
   Content: * 자리물회 ........................................ 7  
* 자리강회 .....................



In [6]:
from src.core import get_bm25_retriever

retriever = get_bm25_retriever()

# BM25Retriever 내부 확인
print("\n=== BM25 내부 상태 ===")
print(f"문서 개수: {len(retriever.docs)}")
print(f"토크나이저 존재: {hasattr(retriever, 'preprocess_func')}")

# 첫 번째 문서의 토큰화 결과 확인
if retriever.docs:
    first_doc = retriever.docs[0].page_content
    print(f"\n첫 문서 원문: {first_doc[:100]}")
    
    # BM25가 실제로 사용한 토큰 확인
    if hasattr(retriever, 'preprocess_func') and retriever.preprocess_func:
        tokens = retriever.preprocess_func(first_doc)
        print(f"BM25 토큰화: {tokens[:20]}")


=== BM25 내부 상태 ===
문서 개수: 148
토크나이저 존재: True

첫 문서 원문: UNESCO logo, Jeju Island UNESCO Global Geopark logo, and MAB logo  
PRINTED WITH SOY INK

# 하늘과 바다가 
BM25 토큰화: ['UNESCO', 'logo,', 'Jeju', 'Island', 'UNESCO', 'Global', 'Geopark', 'logo,', 'and', 'MAB', 'logo', 'PRINTED', 'WITH', 'SOY', 'INK', '#', '하늘과', '바다가', '사랑한', '섬']


In [2]:
# debug_bm25.py
from src.core import (
    get_bm25_retriever, 
    get_korean_bm25_tokenizer,
    clean_query_for_bm25,
    KeywordModifyingRetriever
)

print("="*70)
print("1단계: 토크나이저 테스트")
print("="*70)
tokenizer = get_korean_bm25_tokenizer()
q1 = "고사리육개장"
q2 = "고사리 육개장"
t1 = tokenizer(q1)
t2 = tokenizer(q2)
print(f"'{q1}' → {t1}")
print(f"'{q2}' → {t2}")
print(f"토큰 일치: {t1 == t2}")

print("\n" + "="*70)
print("2단계: clean_query_for_bm25 테스트")
print("="*70)
c1 = clean_query_for_bm25(q1)
c2 = clean_query_for_bm25(q2)
print(f"'{q1}' → '{c1}'")
print(f"'{q2}' → '{c2}'")
print(f"결과 일치: {c1 == c2}")

print("\n" + "="*70)
print("3단계: Raw BM25 직접 검색")
print("="*70)
raw_retriever = get_bm25_retriever()
r1 = raw_retriever.invoke(q1)
r2 = raw_retriever.invoke(q2)
print(f"'{q1}' 검색 결과: {len(r1)}개")
if r1:
    print(f"  - {r1[0].page_content[:]}")
print(f"'{q2}' 검색 결과: {len(r2)}개")
if r2:
    print(f"  - {r2[0].page_content[:]}")

print("\n" + "="*70)
print("4단계: Wrapped BM25 검색")
print("="*70)
wrapped = KeywordModifyingRetriever(
    retriever=raw_retriever,
    formatter=clean_query_for_bm25
)
w1 = wrapped.invoke(q1)
w2 = wrapped.invoke(q2)
print(f"'{q1}' 검색 결과: {len(w1)}개")
if w1:
    print(f"  - {w1[0].page_content[:50]}")
print(f"'{q2}' 검색 결과: {len(w2)}개")
if w2:
    print(f"  - {w2[0].page_content[:50]}")

print("\n" + "="*70)

1단계: 토크나이저 테스트


Quantization is not supported for ArchType::

✅ BM25 Tokenizer 초기화 완료. 불용어 675개 적용됨.
'고사리육개장' → ['고사리', '육개장']
'고사리 육개장' → ['고사리', '육개장']
토큰 일치: True

2단계: clean_query_for_bm25 테스트
'고사리육개장' → '고사리 육개장'
'고사리 육개장' → '고사리 육개장'
결과 일치: True

3단계: Raw BM25 직접 검색
Initializing BM25Retriever (from PGVector's stored documents)...
벡터스토어에서 문서 로드 중...


neon. Fall back to non-quantized model.


✅ 벡터스토어에서 148개 문서 로드 완료
BM25Retriever initialization complete.
'고사리육개장' 검색 결과: 5개
  - * 몸국 ............................................ 18  
* 고사리육개장 .................................... 18  
* 깅이죽 .......................................... 19  
* 말고기 .......................................... 21  
* 빙떡 ............................................ 22  
* 오메기떡 ........................................ 23  
* 향토음식점 리스트 ............................ 24
'고사리 육개장' 검색 결과: 5개
  - * 몸국 ............................................ 18  
* 고사리육개장 .................................... 18  
* 깅이죽 .......................................... 19  
* 말고기 .......................................... 21  
* 빙떡 ............................................ 22  
* 오메기떡 ........................................ 23  
* 향토음식점 리스트 ............................ 24

4단계: Wrapped BM25 검색
'고사리육개장' 검색 결과: 5개
  - * 몸국 ............................................ 
'고사리 육개장' 검색 결과: 5개
  - * 몸국 .................................